In [3]:
import threading

import requests

In [8]:
api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.96&longitude=7.63&hourly=apparent_temperature,rain,windspeed_10m&start_date=2022-12-12&end_date=2022-12-12"

{'latitude': 51.96, 'longitude': 7.64, 'generationtime_ms': 0.3859996795654297, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 69.0, 'hourly_units': {'time': 'iso8601', 'apparent_temperature': '°C', 'rain': 'mm', 'windspeed_10m': 'km/h'}, 'hourly': {'time': ['2022-12-12T00:00', '2022-12-12T01:00', '2022-12-12T02:00', '2022-12-12T03:00', '2022-12-12T04:00', '2022-12-12T05:00', '2022-12-12T06:00', '2022-12-12T07:00', '2022-12-12T08:00', '2022-12-12T09:00', '2022-12-12T10:00', '2022-12-12T11:00', '2022-12-12T12:00', '2022-12-12T13:00', '2022-12-12T14:00', '2022-12-12T15:00', '2022-12-12T16:00', '2022-12-12T17:00', '2022-12-12T18:00', '2022-12-12T19:00', '2022-12-12T20:00', '2022-12-12T21:00', '2022-12-12T22:00', '2022-12-12T23:00'], 'apparent_temperature': [-3.5, -3.4, -3.3, -3.2, -3.1, -3.5, -3.5, -3.5, -3.7, -3.7, -3.3, -2.8, -2.2, -2.3, -2.4, -2.4, -2.6, -3.0, -4.0, -4.4, -4.9, -6.5, -6.6, -7.0], 'rain': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [36]:
from typing import Tuple
import datetime

api_url = "https://api.open-meteo.com/v1/forecast?latitude=51.96&longitude=7.63&hourly=temperature_2m,rain,snowfall,windspeed_10m&daily=weathercode&timezone=Europe%2FBerlin&start_date={start}&end_date={end}"

def get_score(day: datetime.datetime) -> Tuple[float, int, int]:
    formatted = day.strftime("%Y-%m-%d")
    res = requests.get(api_url.format(start=formatted, end=formatted)).json()
    hourly = res["hourly"]

    temp = hourly["temperature_2m"]
    rain = hourly["rain"]
    snowfall = hourly["snowfall"]
    wind = hourly["windspeed_10m"]

    hour = day.hour + (1 if day.minute >= 20 else 0)

    c_temp = temp[hour]
    c_rain = rain[hour]
    c_snow = snowfall[hour]
    c_wind = wind[hour]

    points = 0
    max_points = 0

    max_points += 43
    # in C
    if c_temp < -5:
        points += 43
    elif c_temp < 0:
        points += 3
    elif c_temp < 5:
        points += 2
    elif c_temp < 10:
        points += 1

    max_points += 15
    # in mm
    if c_rain > 30:
        points += 15
    elif c_rain > 10:
        points += 8
    elif c_rain > 2.5:
        points += 4
    elif c_rain > 0:
        points += 1

    max_points += 15
    # in cm
    if c_snow > 10:
        points += 15
    elif c_snow > 3:
        points += 4
    elif c_snow > 1:
        points += 2
    elif c_snow > 0:
        points += 1

    max_points += 15
    # in km/h
    if c_wind > 60: # stürmischer wind
        points += 15
    elif c_wind > 40: # starker wind
        points += 9
    elif c_wind > 30: # frische briese
        points += 3
    elif c_wind > 12: # schwache briese
        points += 1

    return points / max_points, points, max_points

get_score(datetime.datetime.now())

(0.03409090909090909, 3, 88)